In [16]:
! pip install scispacy
! pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_core_sci_lg-0.5.0.tar.gz
! pip install pandas
! pip install requests
! pip install requests pandas
! pip install json
! pip install argparse
! pip install shelve
! pip install scispacy
! pip install scispacy.linking
! pip install datetime
!pip install tabulate

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 532.3 MB 42 kB/s              
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 6.1 MB 10.7 MB/s            


     |████████████████████████████████| 668 kB 63.1 MB/s            
     |████████████████████████████████| 57 kB 12.6 MB/s             
     |████████████████████████████████| 11.3 MB 106.1 MB/s            
     |████████████████████████████████| 47 kB 10.1 MB/s             
  Created wheel for en-core-sci-lg: filename=en_core_sci_lg-0.5.0-py3-none-any.whl size=532569822 sha256=f3b047fcec5b972ee48fc8343ef648a21e60a5e6723b96e5e62e3c4cf73d347d
  Stored in directory: /home/jupyter-parumenon/.cache/pip/wheels/d7/3e/3a/71af6fdd9a698032300759047ad34ca1d138bd481ae51097e5
Successfully built en-core-sci-lg
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.12.2
    Uninstalling typing-extensions-4.12.2:
      Successfully uninstalled typing-extensions-4.12.2
  Attempting uninstall: wasabi
    Found existing installation: wasabi 1.1.3
    Uninstalling wasabi-1.1.3:
      Successfully uninstalled wasabi-1.1.3
  Attempting uninstall: typer
    Found ex

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement shelve (from versions: none)
ERROR: No matching distribution found for shelve
Defaulting to user installation because normal site-packages is not writeable
  Using cached spacy-3.7.5-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.6 MB)
  Using cached thinc-8.2.5-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (937 kB)
  Attempting uninstall: thinc
    Found existing installation: thinc 8.0.17
    Uninstalling thinc-8.0.17:
      Successfully uninstalled thinc-8.

  Attempting uninstall: spacy
    Found existing installation: spacy 3.2.6
    Uninstalling spacy-3.2.6:
      Successfully uninstalled spacy-3.2.6
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-sci-lg 0.5.0 requires spacy<3.3.0,>=3.2.3, but you have spacy 3.7.5 which is incompatible.
Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement scispacy.linking (from versions: none)
ERROR: No matching distribution found for scispacy.linking
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 52 kB 2.5 MB/s             
     |████████████████████████████████| 253 kB 22.3 MB/s            


In [1]:
import spacy
nlp = spacy.load("en_core_sci_lg")  
print("Model loaded successfully!")

/home/jupyter-parumenon/.local/lib/python3.9/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_sci_lg' (0.5.0) was trained with spaCy v3.2.3 and may not be 100% compatible with the current version (3.7.5). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


Model loaded successfully!


## Apply spacy model, extract entities, get cuis and then crosswalk to icd10 codes for Eg text

In [2]:
#import all libraries
import spacy
import scispacy
from scispacy.linking import EntityLinker
import pandas as pd
from tabulate import tabulate
from IPython.display import display  # Import display function for Jupyter

# Load CSV containing UMLS_CUI and icd10 code mappings. These are extracted from umls mrconso file that is updated twice a year
csv_file_path = "umls_cuis.csv" 
code_df = pd.read_csv(csv_file_path, encoding="latin1")
print("File loaded successfully!")

# Load the large model with vector embeddings 
nlp = spacy.load("en_core_sci_lg")

# Adding the UMLS entity linker
nlp.add_pipe("scispacy_linker", config={"resolve_abbreviations": True, "linker_name": "umls"})

print("Model loaded successfully!")

# input for our test case
text = "Patient presents with chest pain and nausea. He has a history of hypertension and type 2 diabetes."

# Applying the NLP model
doc = nlp(text)

# Initialize a list to hold all entities and their UMLS mappings
entities = []

# Extract entities and their UMLS links
for ent in doc.ents:
    print(f"Entity: {ent}, Label: {ent.label_}")

    # For each entity, look at the linked UMLS concepts
    for umls_ent in ent._.kb_ents:
        linker = nlp.get_pipe("scispacy_linker")
        concept = linker.kb.cui_to_entity[umls_ent[0]]
        
        # Extract the score for the UMLS entity match
        score = umls_ent[1]  # This contains the matching score
        
        # Append the entity details along with the score
        entities.append({
            "Text": ent.text,
            "Start": ent.start_char,
            "End": ent.end_char,
            "Label": ent.label_,
            "UMLS_CUI": concept.concept_id,
            "UMLS_Name": concept.canonical_name,
            "Score": score  # Add the score to the output
        })

# Convert the entities list to a DataFrame for easier viewing
entities_df = pd.DataFrame(entities)
print("Entities Created")

# Improve display settings for Jupyter
pd.set_option("display.max_columns", None)  # Show all columns
pd.set_option("display.max_rows", 50)  # Adjust number of rows displayed
pd.set_option("display.width", 1000)  # Prevent line wrapping

# Display the entities_df as a formatted table
display(entities_df.style.set_properties(**{"text-align": "left"}).set_table_styles(
    [{"selector": "th", "props": [("text-align", "left")]}]
))

# Rename "cui" to "UMLS_CUI" for consistency in umls-icd10 CSV file to match
#code_df.rename(columns={"cui": "UMLS_CUI"}, inplace=True)  

# Merge entities_df with the CSV file on "UMLS_CUI"
merged_df = entities_df.merge(code_df, on="UMLS_CUI", how="left")  

# Select the row with the highest Score for each entity- to reduce 1:many matches- this is optional
best_matches_df = merged_df.loc[merged_df.groupby("Text")["Score"].idxmax()]


# Display the final DataFrame as a formatted table
display(best_matches_df.style.set_properties(**{"text-align": "left"}).set_table_styles(
    [{"selector": "th", "props": [("text-align", "left")]}]
))

Your CPU supports instructions that this binary was not compiled to use: SSE3 SSE4.1 SSE4.2 AVX AVX2
For maximum performance, you can install NMSLIB from sources 
pip install --no-binary :all: nmslib
/home/jupyter-parumenon/.local/lib/python3.9/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_sci_lg' (0.5.0) was trained with spaCy v3.2.3 and may not be 100% compatible with the current version (3.7.5). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


File loaded successfully!


/home/jupyter-parumenon/.local/lib/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator TfidfTransformer from version 1.1.2 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/jupyter-parumenon/.local/lib/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator TfidfVectorizer from version 1.1.2 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Model loaded successfully!
Entity: Patient, Label: ENTITY
Entity: chest pain, Label: ENTITY
Entity: nausea, Label: ENTITY
Entity: history, Label: ENTITY
Entity: hypertension, Label: ENTITY
Entity: type 2 diabetes, Label: ENTITY
Entities Created


,Text,Start,End,Label,UMLS_CUI,UMLS_Name,Score
0,Patient,0,7,ENTITY,C0030705,Patients,0.982997
1,Patient,0,7,ENTITY,C1550655,Specimen Type - Patient,0.982997
2,Patient,0,7,ENTITY,C1578481,Mail Claim Party - Patient,0.982997
3,Patient,0,7,ENTITY,C1578483,Report source - Patient,0.982997
4,Patient,0,7,ENTITY,C1578484,Relationship modifier - Patient,0.982997
5,chest pain,22,32,ENTITY,C0008031,Chest Pain,0.984917
6,chest pain,22,32,ENTITY,C2926613,Chest pain:Finding:Point in time:^Patient:Ordinal,0.984917
7,chest pain,22,32,ENTITY,C4054856,Have Chest Pain,0.794442
8,nausea,37,43,ENTITY,C0027497,Nausea,0.995102
9,nausea,37,43,ENTITY,C4255480,Nausea:Presence or Threshold:Point in time:^Patient:Ordinal,0.995102


,Text,Start,End,Label,UMLS_CUI,UMLS_Name,Score,ICD10_code,term_type,coding_system,ICD10_description
0,Patient,0,7,ENTITY,C0030705,Patients,0.982997,nan,nan,nan,nan
5,chest pain,22,32,ENTITY,C0008031,Chest Pain,0.984917,R07.9,PT,ICD10CM,"Chest pain, unspecified"
13,history,54,61,ENTITY,C0019664,History,0.994278,nan,nan,nan,nan
18,hypertension,65,77,ENTITY,C0020538,Hypertensive disease,0.983864,I10,ET,ICD10CM,high blood pressure
8,nausea,37,43,ENTITY,C0027497,Nausea,0.995102,R11.0,PT,ICD10CM,Nausea
23,type 2 diabetes,82,97,ENTITY,C0011860,"Diabetes Mellitus, Non-Insulin-Dependent",0.984797,E11,AB,ICD10CM,Type 2 diabetes mellitus


## How to navigate umls mrconso file to get as much as possible 1:1 mappings 

In [ ]:
#step 1 download UMLS MRCONSO file (2GB) from their site and upload into postgre DB. 

#This is updated twice a year so minimal maintainance

#them look at distinct sab and tty combinations to establish priority

/*
PT	ICD10CM	- PREF TERM
AB	ICD10CM - ABBREVIATION
ET	ICD10CM - Entry term
HT	ICD10CM - Hierarchical term
PT	ICD10- PREF TERM
HT	ICD10 - Hierarchical term
PS	ICD10 - Short forms that needed full specification
PX	ICD10 - Short forms that needed full specification
HX	ICD10 - Expanded version of short hierarchical term
HS	ICD10 - Short or alternate version of hierarchical term
PT	SNOMEDCT - PREF TERM 
FN	SNOMEDCT - Full form of descriptor 
SY	SNOMEDCT - Synonym 
PTGB	SNOMEDCT - British preferred term 
SYGB	SNOMEDCT - British synonym 
MTH_PT	SNOMEDCT - Metathesaurus preferred term 
MTH_FN	SNOMEDCT - MTH Full form of descriptor 
MTH_SY	SNOMEDCT - MTH Designated synonym 
MTH_PTGB	SNOMEDCT - Metathesaurus-supplied form of British preferred term 
SB	SNOMEDCT - Named subset of a source 
MTH_SYGB	SNOMEDCT - Metathesaurus-supplied form of British synonym 
XM	SNOMEDCT - other 
*/
#I did this locally as this is 2 gb file.
WITH RankedConcepts AS (
    SELECT 
        cui, 
        code, 
        tty,
        sab,
        ROW_NUMBER() OVER (
            PARTITION BY cui 
            ORDER BY 
                CASE 
                    WHEN sab = 'ICD10CM' AND tty = 'PT' THEN 1  -- Highest priority
                    WHEN sab = 'ICD10CM' AND tty = 'AB' THEN 2  
                    WHEN sab = 'ICD10CM' AND tty = 'ET' THEN 3  
                    WHEN sab = 'ICD10CM' AND tty = 'HT' THEN 4  
                    WHEN sab = 'ICD10' AND tty = 'PT' THEN 5  
                    WHEN sab = 'ICD10' AND tty = 'HT' THEN 6  
                    WHEN sab = 'ICD10' AND tty IN ('PS', 'PX') THEN 7  
                    WHEN sab = 'ICD10' AND tty = 'HX' THEN 8  
                    WHEN sab = 'ICD10' AND tty = 'HS' THEN 9
                    WHEN sab = 'SNOMEDCT_US' AND tty = 'PT' THEN 10
                    WHEN sab = 'SNOMEDCT_US' AND tty = 'FN' THEN 11
                    WHEN sab = 'SNOMEDCT_US' AND tty = 'SY' THEN 12
                    WHEN sab = 'SNOMEDCT_US' AND tty = 'PTGB' THEN 13
                    WHEN sab = 'SNOMEDCT_US' AND tty = 'SYGB' THEN 14
                    WHEN sab = 'SNOMEDCT_US' AND tty = 'MTH_PT' THEN 15
                    WHEN sab = 'SNOMEDCT_US' AND tty = 'MTH_FN' THEN 16
                    WHEN sab = 'SNOMEDCT_US' AND tty = 'MTH_SY' THEN 17
                    WHEN sab = 'SNOMEDCT_US' AND tty = 'MTH_PTGB' THEN 18
                    WHEN sab = 'SNOMEDCT_US' AND tty = 'SB' THEN 19
                    WHEN sab = 'SNOMEDCT_US' AND tty = 'MTH_SYGB' THEN 20
                    WHEN sab = 'SNOMEDCT_US' AND tty = 'XM' THEN 21
                    ELSE 25  -- Catch-all for any other TTY
                END
        ) AS rank
    FROM mrconso 
    WHERE sab IN ('ICD10CM', 'ICD10', 'SNOMEDCT_US')  -- Only consider ICD10CM and ICD10 sources
    and code !~ ('-') and suppress != 'O'
)
SELECT cui, code, str, tty, sab 
FROM RankedConcepts 
WHERE rank = 1;  -- Select only the highest-priority row per CUI

## Getting primary diagnosis from MIMIC-IV Notes

In [2]:
import pandas as pd
import re
#loading clinical notes from mimic
csv_file_path = "/data0/mimic-iv-note/2.2/note/discharge.csv" 
df_discharge = pd.read_csv(csv_file_path)
print("notes loaded")

notes loaded


In [3]:
#PART 1 : GETTING THE PRIMARY DIAGNOSIS OUT OF THE NOTES
#Step 1: Define section keywords (you can adjust these based on your specific needs)
sections = {
    "discharge_diag": ["Discharge Diagnosis", "___ Diagnosis:"],
    "discharge_cond": ["Discharge Condition:"]
}


#Step 2: Initialize a list to store extracted DataFrames for each note
extracted_dfs = []

# Iterate over each clinical note in the DataFrame
for index, row in df_discharge.iterrows():
    clinical_notes = row['text']  # Get the clinical note text
    subject_id = row['subject_id']  # Get subject_id
    hadm_id = row['hadm_id']  # Get hadm_id
    
    # Clean up newlines and replace them with spaces
    clinical_notes = clinical_notes.replace("\n", "||")

    # Initialize a dictionary to store extracted data
    extracted_data = {key: "" for key in sections.keys()}
    
    # Iterate over each section to extract relevant data
    for section, phrases in sections.items():
        # Find the start position of the current section keyword
        for phrase in phrases:
            start_index = clinical_notes.lower().find(phrase.lower())
            if start_index != -1:
                # Determine where to stop extracting
                # Look for the next section keyword
                end_index = len(clinical_notes)  # Default to the end of the note
                for next_section in sections.keys():
                    if next_section != section:
                        for next_phrase in sections[next_section]:
                            next_start_index = clinical_notes.lower().find(next_phrase.lower(), start_index)
                            if next_start_index != -1 and next_start_index < end_index:
                                end_index = next_start_index
                # Extract the section text
                extracted_data[section] += clinical_notes[start_index:end_index].strip() + " "
                break  # Break after finding the first match for this section

    # Clean up extracted data to remove unwanted characters
    for key in extracted_data.keys():
        # Remove unwanted characters like "#" and "="
        extracted_data[key] = re.sub(r"[#=]", "", extracted_data[key])
        # Remove empty lines and trim whitespace
        cleaned_text = "\n".join([line.strip() for line in extracted_data[key].splitlines() if line.strip()])
        extracted_data[key] = cleaned_text
    
    # Add subject_id and hadm_id to the extracted data
    extracted_data['subject_id'] = subject_id
    extracted_data['hadm_id'] = hadm_id
    #extracted_data['future_admission_count'] = future_admission_count

    # Convert the extracted data for this note to a DataFrame and add to the list
    extracted_dfs.append(pd.DataFrame([extracted_data]))

print("extraction completed")    
# Step 3: Concatenate all extracted DataFrames
df_with_sections = pd.concat(extracted_dfs, ignore_index=True)

# Step 4: Drop specified columns from the final DataFrame
columns_to_drop = [ 'discharge_cond']
df_with_sections.drop(columns=columns_to_drop, inplace=True, errors='ignore')

# Step 5: Define phrases to remove per column in a dictionary to clean th data as much as possible. you can awalys add to this if you find more.
phrases_to_remove_per_column = {
    "discharge_diag": ["Discharge Diagnosis:", "___", "PRIMARY DIAGNOSES", "Primary diagnosis:", "Primary:", 
                       "Primary Diagnosis", "Primary Diagnosis:", "PRIMARY:", "PRIMARY DIANGOSIS", "Primary Diagnoses:", "PRIMARY", "PRIMARY DIAGNOSIS",
                       "DIAGNOSIS", "DIAGNOSIS:", "SECONDARY DIAGNOSES", "Secondary:", "--------------------", "PREOPERATIVE DIAGNOSIS:", "POSTOPERATIVE DIAGNOSIS:", "Diagnosis",
                       "Secondary diagnosis:", "primary diagnosis:", "secondary diagnosis:", "Primary diagnosis", "Secondary diagnosis", "=", "-", ":", "SECONDARY :", "SECONDARY",
                       "*", "Secondary :", "Secondary Diagnoses", "Secondary Diagnoses:", "Diagnoses"]
    # Add other columns and their specific phrases here
}
print("cleaning completed")  
# Step 6: Iterate through each column and its phrases
for col, phrases in phrases_to_remove_per_column.items():
    if col in df_with_sections.columns:
        for phrase in phrases:
            # Replace each phrase with an empty string and strip whitespace
            df_with_sections[col] = df_with_sections[col].str.replace(phrase, "", regex=False).str.strip()

# Improve display settings for Jupyter
#pd.set_option("display.max_columns", None)  # Show all columns
#pd.set_option("display.max_rows", 50)  # Adjust number of rows displayed
#pd.set_option("display.width", 1000)  # Prevent line wrapping

# Display the after extraction
#display(df_with_sections.style.set_properties(**{"text-align": "left"}).set_table_styles(
    #[{"selector": "th", "props": [("text-align", "left")]}]
#))

df_with_sections.head(20)

extraction completed
cleaning completed


,discharge_diag,subject_id,hadm_id
0,||Ascites from Portal HTN|||| ||,10000032,22595853
1,|| diuretic refractory ascites|| HCV cirrhosi...,10000032,22841357
2,||||Hypotension||Hyperkalemia||Acute Kidney In...,10000032,29079034
3,|| ||||Decompensated HCV cirrhosis||Hyperkalem...,10000032,25742920
4,|| Dementia|| ||,10000084,23052089
5,|| ||dysphagia and foreign body sensation|||| ...,10000117,22927623
6,||Left valgus impacted femoral neck fracture||...,10000117,27988844
7,"|||| Acute muscular hematoma, right flank|| He...",10000248,20600184
8,||renal cell carcinoma|||| ||,10000560,28979390
9,||Nasal fracture||Epistaxis||NSTEMI|||| ||,10000764,27897940


In [4]:
##PART 2: GET COUNTS OF THIS RAW EXTRACTION
print(len(df_with_sections))

print( df_with_sections['subject_id'].nunique() ) 

331794
145915


In [5]:
##PART 3: TRANSFORM INTO A CLEAN MAPPABLE FORMAT AND DROP DUPLICATES

# Create a copy of the EXTRACTED DataFrame
df_transformed = df_with_sections.copy()

# Split 'discharge_diag' on '|' and explode into separate rows
df_transformed['discharge_diag'] = df_transformed['discharge_diag'].str.split('|')
df_transformed = df_transformed.explode('discharge_diag')

# Remove any empty strings or spaces that might result from splitting
df_transformed['discharge_diag'] = df_transformed['discharge_diag'].str.strip()

# Drop rows where 'discharge_diag' is empty or contains only punctuation
df_transformed = df_transformed[df_transformed['discharge_diag'].str.contains(r'[a-zA-Z0-9]', regex=True)]

# Convert discharge_diag to lowercase for case-insensitive comparison
df_transformed["discharge_diag_lower"] = df_transformed["discharge_diag"].str.lower()

# Drop duplicate diagnoses within the same subject_id
df_transformed = df_transformed.drop_duplicates(subset=["subject_id", "discharge_diag_lower"], keep="first")

# Drop the temporary lowercase column
df_transformed = df_transformed.drop(columns=["discharge_diag_lower"])


df_transformed = df_transformed.reset_index(drop=True)

# Improve display settings for Jupyter
#pd.set_option("display.max_columns", None)  # Show all columns
#pd.set_option("display.max_rows", 50)  # Adjust number of rows displayed
#pd.set_option("display.width", 1000)  # Prevent line wrapping

# Display df_transformed with improved formatting
#display(df_transformed.style.set_properties(**{"text-align": "left"}).set_table_styles(
    #[{"selector": "th", "props": [("text-align", "left")]}]
#))

df_transformed.head(20)

#save to csv
#df_transformed.to_csv("structured_diagnosis")

,discharge_diag,subject_id,hadm_id
0,Ascites from Portal HTN,10000032,22595853
1,diuretic refractory ascites,10000032,22841357
2,"HCV cirrhosis, HIV, hyponatremia, COPD",10000032,22841357
3,Hypotension,10000032,29079034
4,Hyperkalemia,10000032,29079034
5,Acute Kidney Injury,10000032,29079034
6,HIV,10000032,29079034
7,Cirrhosis,10000032,29079034
8,COPD,10000032,29079034
9,Decompensated HCV cirrhosis,10000032,25742920


In [6]:
##PART 4 GET COUNTS AFTER TRANSFORMATION
print(len(df_transformed))

print( df_transformed['subject_id'].nunique() ) 

1086388
145352


##USING THE TRANSFORMED DIAGNOSIS FROM NOTES TO MAP WITH SPACY MODEL- Contnued in part 2 file